In [1]:
!ls data/han/AWS/

남이섬_2016.xlsx  남이섬_2020.xlsx  청평_2019.xlsx  화천_2018.xlsx
남이섬_2017.xlsx  청평_2016.xlsx    청평_2020.xlsx  화천_2019.xlsx
남이섬_2018.xlsx  청평_2017.xlsx    화천_2016.xlsx  화천_2020.xlsx
남이섬_2019.xlsx  청평_2018.xlsx    화천_2017.xlsx


In [2]:
!pwd

/home/cjinw/work/water-quality/rnn


In [3]:
%matplotlib widget
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt
    
from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras

In [4]:
# for Korean in plot
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [6]:

# folder = 'data/han/자동'
# folder = 'data/han/수질'
# folder = 'data/han/AWS'
folder = 'data/nakdong/자동'
# folder = 'data/gum/자동'
# folder = 'data/yeongsan1/자동'

# ## 한강
# ## 자동
# file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], 
#               ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx'],
#               ['서상_2016.xlsx','서상_2017.xlsx','서상_2018.xlsx', '서상_2019.xlsx']]

## 수질
# file_names = [['가평천3_2016.xlsx','가평천3_2017.xlsx','가평천3_2018.xlsx', '가평천3_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천댐1_2016.xlsx','춘천댐1_2017.xlsx','춘천댐1_2018.xlsx', '춘천댐1_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['의암_2016.xlsx','의암_2017.xlsx','의암_2018.xlsx', '의암_2019.xlsx'],
#               ['춘성교_2016.xlsx','춘성교_2017.xlsx','춘성교_2018.xlsx', '춘성교_2019.xlsx'],
#               ['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천_2016.xlsx','춘천_2017.xlsx','춘천_2018.xlsx', '춘천_2019.xlsx']
#              ]


## AWS
# file_names = [['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2018.xlsx'],
#               ['청평_2016.xlsx', '청평_2017.xlsx', '청평_2018.xlsx', '청평_2019.xlsx'],
#               ['화천_2016.xlsx', '화천_2017.xlsx', '화천_2018.xlsx', '화천_2019.xlsx']]
 

    
## 낙동강
# #자동
file_names = [['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx'],
              ['신암_2016.xlsx','신암_2017.xlsx','신암_2018.xlsx', '신암_2019.xlsx'],
              ['회상_2016.xlsx','회상_2017.xlsx','회상_2018.xlsx', '회상_2019.xlsx']]


## 금강
#자동
# file_names = [['현도_2016.xlsx','현도_2017.xlsx','현도_2018.xlsx', '현도_2019.xlsx'],
#               ['대청호_2016.xlsx','대청호_2017.xlsx','대청호_2018.xlsx', '대청호_2019.xlsx'],
#               ['옥천천_2016.xlsx','옥천천_2017.xlsx','옥천천_2018.xlsx', '옥천천_2019.xlsx'],
#               ['장계_2016.xlsx','장계_2017.xlsx','장계_2018.xlsx', '장계_2019.xlsx']]
             

    
    
## 영산강
#자동
# file_names = [['나주_2016.xlsx','나주_2017.xlsx','나주_2018.xlsx', '나주_2019.xlsx'],
#               ['지석천_2016.xlsx','지석천_2017.xlsx','지석천_2018.xlsx', '지석천_2019.xlsx'],
#               ['용봉_2016.xlsx','용봉_2017.xlsx','용봉_2018.xlsx', '용봉_2019.xlsx'],
#               ['서창교_2016.xlsx','서창교_2017.xlsx','서창교_2018.xlsx', '서창교_2019.xlsx'],
#               ['우치_2016.xlsx','우치_2017.xlsx','우치_2018.xlsx', '우치_2019.xlsx']]
              

In [7]:
target_col = '용존산소'
# 각각의 강 마다, 측정소 별로 column을 맞춰야함
# 0: 자동, 1: 수질, 2: AWS(ASOS), 
han = [':,2:', ':,28:45', ':,26:29']
nakdong = [':,2:', ':,2:12']
gum = [':,2:'] #, ':,28:45', ':,26:29']
# gain_han = ['save/han/1', 'save/han/2', 'save/han/3']

# interpolation_option = True
interpolation_option = False
# gain_save_path = gain_han[2]

iloc_val = gum[0]
iloc_val

':,2:'

In [8]:
def make_columns(df):
    column_list = ['측정날짜', '측정소명', '수온', '수소이온농도','전기전도도', '용존산소', '총유기탄소', '총질소', '총인', '클로로필-a']
    list_df = pd.DataFrame(columns=column_list)
    list_df
    df = df.drop(columns=df.columns.difference(column_list))
    new_column = list_df.columns.difference(df.columns)
#     print(new_column)
    if not new_column.empty :
        print("Make_columns")
        for i in range(new_column.shape[0]):
            df[new_column[i]] = pd.Series()
#     print('columns')
#     print(df.columns)
    return df


In [9]:
def make_timeseries(df, interpolate=None):
    print(df.shape)
    date_col = df.columns[0]
    df[date_col] = pd.to_datetime(df[date_col])
    df = df[df[date_col].notna()]

    year = pd.DatetimeIndex(df[date_col]).year.astype(np.int64)

    start = str(year[0]) + "-01-01 00:00"
#     start

    end = str(year[0]) + "-12-31 23:00"
#     end

    time_series = pd.date_range(start = start, end = end, freq='H')
    time_series
    time_series = pd.DataFrame(time_series)
    time_series.columns = [date_col]   
    time_series

    df = pd.concat([time_series, df], axis=0)
    df = df.drop_duplicates([date_col], keep="last")
    df = df.sort_values([date_col], axis=0)
    
    if interpolate :
        print('interpolation')
        df = df.interpolate(metod='spline', order=2, limit_direction='both')
        
    print(df.shape)
    return df

In [10]:
def make_dataframe(file_names, iloc_val, interpolate=None):
    day = 24*60*60
    year = (365.2425)*day

    df_full = []
    df = []
    
    for loc in range(len(file_names)):
    
        df_loc = []
        for y in range(len(file_names[loc])):
            path = os.path.join(folder, file_names[loc][y])
            print(file_names[loc][y])
            df_tmp = make_timeseries(pd.read_excel(path), interpolate=interpolate)
#             df_tmp = pd.read_excel(path).dropna(axis='columns', how = 'all')
            df_tmp = make_columns(df_tmp)
            
    #         df_tmp = pd.read_excel(path).dropna(axis='columns', how = 'all')
    #         print(df_tmp.head) 
            df_loc.append(df_tmp)
#             df_loc.append(pd.read_excel(path))
        df_full.append(pd.concat(df_loc))
    #     df.append(df_full[loc].iloc[:,2:11])
        df.append(eval(f"df_full[loc].iloc[{iloc_val}]"))
    #     df.append(df_full[loc].iloc[:, [2,3,4,5,6,7,10]])
    #     df.append(df_full[loc].iloc[:, 2:11])
        date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
        timestamp_s = date_time.map(datetime.datetime.timestamp)
        df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
        df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
        df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
        df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
        df[loc] = df[loc].reset_index(drop=True)
        
#         print(df[loc].shape[0])
#         rows = df[loc].shape[0]
#         percentage = round(rows * 0.7)
#         df[loc] = df[loc].dropna(axis='columns',  thresh=percentage)
        

    
    return df

In [11]:
def normalize(df):
    # normalize data
    df_all = pd.concat(df)
    
    train_mean = df_all.mean()
    train_std = df_all.std()
    for i in range(len(file_names)):
        df[i] = (df[i]-train_mean)/train_std
    
    
    return df_all, train_mean, train_std

In [12]:
def min_max_normalize(df):
    # normalize data
    df_all = pd.concat(df)
    
    train_min = df_all.min()
    train_max = df_all.max()
    for i in range(len(file_names)):
        df[i] = (df[i]-train_min)/(train_max - train_min)       
    
    
    return df_all, train_min, train_max

In [13]:
##for normal
# df = make_dataframe(file_names, iloc_val)
## for interpolation
df = make_dataframe(file_names, iloc_val, interpolate=interpolation_option)
df_all, train_mean, train_std = normalize(df)
# df_all, train_mean, train_std = min_max_normalize(df)

도개_2016.xlsx
(8784, 12)
(8784, 12)
도개_2017.xlsx
(8760, 12)
(8760, 12)
도개_2018.xlsx
(8755, 12)
(8760, 12)
도개_2019.xlsx
(8760, 12)
(8760, 12)
신암_2016.xlsx
(8742, 11)
(8784, 11)
신암_2017.xlsx
(8744, 11)
(8760, 11)
신암_2018.xlsx
(8758, 11)
(8760, 11)
신암_2019.xlsx
(8759, 11)
(8760, 11)
회상_2016.xlsx
(8784, 27)
(8784, 27)
회상_2017.xlsx
(8742, 27)
(8760, 27)
회상_2018.xlsx
(8760, 27)
(8760, 27)
회상_2019.xlsx
(8760, 27)
(8760, 27)


In [14]:
train_mean, train_std

(수온          1.620074e+01
 수소이온농도      7.959209e+00
 전기전도도       2.361947e+02
 용존산소        1.029059e+01
 총유기탄소       2.949737e+00
 총질소         2.511964e+00
 총인          1.931998e-02
 클로로필-a      1.822085e+01
 Day sin     6.037103e-15
 Day cos     2.747682e-15
 Year sin   -5.681266e-08
 Year cos    2.053380e-05
 dtype: float64,
 수온           9.217941
 수소이온농도       0.600598
 전기전도도       44.257904
 용존산소         2.455950
 총유기탄소        0.885916
 총질소          0.791127
 총인           0.022864
 클로로필-a      14.528678
 Day sin      0.707110
 Day cos      0.707110
 Year sin     0.707103
 Year cos     0.707117
 dtype: float64)

## 

In [15]:
# just for Create WidnowGenerator 
train_df = df[0]
val_df = df[0]
test_df = df[0]

In [16]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
#                out_features = None,
               out_features = None, out_num_features = None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [17]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [18]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [19]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result
@property
def example2(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example2', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.val))
    # And cache it for next time
    self._example = result
  return result
@property
def example3(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example3', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.test))
    # And cache it for next time
    self._example = result
  return result


WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example
WindowGenerator.example2 = example2
WindowGenerator.example3 = example3

In [20]:
def sample_batch_index(total, batch_size):
    '''Sample index of the mini-batch.

    Args:
        - total: total number of samples
        - batch_size: batch size

    Returns:
        - batch_idx: batch index
    '''
    total_idx = np.random.permutation(total)
    batch_idx = total_idx[:batch_size]
    return batch_idx

In [21]:
def binary_sampler(p, shape):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - shape: matrix shape
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = shape)
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [22]:
def uniform_sampler(low, high, shape):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = shape)

In [23]:
class MissData(object):
    def __init__(self, load_dir=None):
        if load_dir:
            self.missarr = np.load(os.path.join(load_dir, 'miss.npy'))
            self.idxarr = np.load(os.path.join(load_dir, 'idx.npy'))
            
    def make_missdata(self, data_x, missrate=0.2):
        data = data_x.copy()
        rows, cols = data_x.shape
        total_no = rows*cols
        total_miss_no = np.round(total_no*missrate).astype(int)
        total_idx = self.idxarr.shape[0]
        idxarr = self.idxarr
        missarr = self.missarr
        #print(total_miss_no)
        miss_no = 0
        cum_no = self.idxarr[:,3:4]
        cum_no = cum_no.reshape((total_idx))
        cum_sum = np.max(cum_no)
        #print(cum_no)
        #print(total_idx)
        while True:
            loc_count = np.around(np.random.random()*cum_sum)
            #print('loc_count =', loc_count)
            idx = len(cum_no[cum_no <= loc_count])-1
            #print(cum_no[cum_no <= loc_count])
            #print('idx =', idx)
            startnan = idxarr[idx][0]
            nanlen = idxarr[idx][2]
            loc = np.around(np.random.random()*(rows-nanlen)).astype(int)
            #print('loc =', loc)
            #print(loc_count, idx)
            #print(idxarr[idx])
            #data_copy = data[loc:loc+nanlen].copy()
            data_copy = data[loc:loc+nanlen]
            #print('startnan=', startnan)
            #isnan = missarr[startnan:startnan+nanlen].copy()
            isnan = missarr[startnan:startnan+nanlen]
            #print('isnan =',isnan)
            miss_no += idxarr[idx][1]
            if (miss_no > total_miss_no):
                break
            data_copy[isnan==1] = np.nan
            data[loc:loc+nanlen] = data_copy
        #print('miss_data =', data)
        return data
    
    def save(data, max_tseq, save_dir='save'):
        no, dim = data.shape
        #print((no, dim))
        isnan = np.isnan(data).astype(int)
        isany = np.any(isnan, axis=1).astype(int)
        shifted = np.roll(isany, 1)
        shifted[0] = 1
        #print(isnan)
        #print(isany.astype(int))
        #print(shifted)
        startnan = ((isany == 1) & (shifted ==0)).astype(int)
        #print(startnan)
        group = startnan.cumsum()
        group = group*isany
#         print(group)
        n = np.max(group)
#         print(n)
        missarr = None
        cum_no = 0
        rowidx = 0
        for i in range(1, n+1):
            g = (group == i).astype(int)
            i = np.argmax(g)
            rows = g.sum()
            #print(len)
            #print(i)
            #print(type(missarr))
            if rows <= max_tseq:
                nanseq = isnan[i:i+rows, :]
                no = np.sum(nanseq)
                #print(no)
                if missarr is None:
                    missarr = nanseq
                    idxarr = np.array([[rowidx, no, rows, cum_no]])
                else:
                    missarr = np.concatenate((missarr, nanseq))
                    idxarr = np.concatenate((idxarr, [[rowidx, no, rows, cum_no]]), axis=0)
                cum_no += no
                rowidx += rows

        print(idxarr)
        miss_npy_file = os.path.join(save_dir, 'miss.npy')
        idx_npy_file = os.path.join(save_dir, 'idx.npy')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        np.save(miss_npy_file, missarr)
        np.save(idx_npy_file, idxarr)
        print('miss_data file saved')

In [24]:
def interpolate(np_data, max_gap=3):
    #n = np_data.shape[1]
    data = pd.DataFrame(np_data)
    #data[0][0] = np.nan
    #data[0][1] = np.nan
    #data[0][2] = np.nan
    #data[data.columns[0]][0] = np.nan
    #data[data.columns[0]][1] = np.nan
    #data[data.columns[0]][2] = np.nan
    
    # create mask
    mask = data.copy()
    grp = ((mask.notnull() != mask.shift().notnull()).cumsum())
    grp['ones'] = 1
    for i in data.columns:
        mask[i] = (grp.groupby(i)['ones'].transform('count') < max_gap) | data[i].notnull()
    data = data.interpolate(method='polynomial', order=5, limit=max_gap, axis=0).bfill()[mask]
    return data.to_numpy()
    #return data
    
#filled_data = interpolate(norm_data, max_gap=3)
#np.arange(0, 5, dtype=int)
#['%d'%val for val in range(0,5)]

In [25]:
class GainDataGenerator(keras.utils.Sequence):
    'Generates data for GAIN'
    def __init__(self,
                 data_list,
                 batch_size=32,
                 input_width=24*3,
                 label_width=24*3,
                 shift=0,
                 fill_no=4,
                 miss_rate=0.2,
                 hint_rate=0.9,
                 normalize=True,
                 miss_pattern=None,
                 miss_data_load_dir='save',
                 alpha=100.):
        'Initialization'
        window_size = input_width
        
        # interpollation
        filled_data = []
        for data in data_list:
            data = interpolate(data, max_gap=fill_no)
            filled_data.append(data)
            
        data_list = filled_data
        
        # whole data
        self.data = np.concatenate(data_list)

        # TO-DO
        
        # pre calculation for  sequence data
        last_cum = 0
        cums = []
        for data in data_list:
            isnan = np.isnan(data)
            isany = np.any(isnan, axis=1)
            #shift same as pd.shift(isany, fill_value=True)
            shifted = np.roll(isany, 1)
            shifted[0] = True # set to nan
            
            start_seq = ((isany == False) & (shifted == True)).astype(int)
            cum = start_seq.cumsum()
            cum += last_cum
            last_cum = np.max(cum)
            cum[isany] = 0
            cums.append(cum)

        # Define mask matrix
        if miss_pattern is None:
            print("pattern none")
            self.data_m = binary_sampler(1-miss_rate, self.data.shape)
        else:
            #MissData.save(self.data, max_tseq = 12)
            print("load save")
            self.miss = MissData(load_dir=miss_data_load_dir)
            self.miss_rate = miss_rate
            miss_data = self.miss.make_missdata(self.data, self.miss_rate)
            self.data_m = 1. - np.isnan(miss_data).astype(float)
            
            self.data_m_rand = binary_sampler(1-(miss_rate/10.), self.data.shape)
            self.data_m[self.data_m_rand==0.] = 0.
        self.miss_pattern = miss_pattern
        
        # sequence data
        self.ids = np.concatenate(cums)
        data_idx = np.empty((0), dtype=int)
        for i in range(1, last_cum+1):
            seq_len = (self.ids == i).sum()
            start_id = np.argmax(self.ids == i)
            # possible data number in seqeunce
            time_len = seq_len - window_size + 1
            start_ids = np.arange(start_id, start_id+time_len)
            data_idx = np.append(data_idx, start_ids)
            
        # start index set for sequence data
        self.data_idx = data_idx
        self.input_width = input_width
        self.no = len(data_idx)
        
        #print('self.no = ', self.no)
        
        self.batch_size = batch_size
        
        # random shuffling  index
        self.batch_idx = sample_batch_index(self.no, self.no)
        self.batch_id = 0
        self.shape = (batch_size,self.input_width)+self.data.shape[1:]
        #self.hint_rate = hint_rate
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        # Sample batch
        x = np.empty((0, self.input_width, self.data.shape[1]))
        #m = np.empty((0, self.input_width, self.data.shape[1]))
        #h = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.input_width, self.data.shape[1]))
        #print(x.shape)
        #print(self.data.shape)
        #print(self.input_width)
        #self.batch_idx = sample_batch_index(self.no, self.batch_size)
        for cnt in range(0, self.batch_size):
            i = self.batch_idx[self.batch_id]
            self.batch_id += 1
            #self.batch_id %= self.batch_size
            self.batch_id %= self.no
            if self.miss_pattern and (self.batch_id == 0):
                self.batch_idx = sample_batch_index(self.no, self.no)
                miss_data = self.miss.make_missdata(self.data, self.miss_rate)
                self.data_m = 1. - np.isnan(miss_data).astype(float)
                self.data_m_rand = binary_sampler(1-self.miss_rate/10., self.data.shape)
                self.data_m[self.data_m_rand==0.] = 0.
            idx1 = self.data_idx[i]
            idx2 = self.data_idx[i]+self.input_width
            #print(idx1, idx2)
        
            Y_mb = self.data[idx1:idx2].copy()
            X_mb = Y_mb.copy()
            M_mb = self.data_m[idx1:idx2]
            Z_mb = uniform_sampler(0, 0.01, shape=X_mb.shape)
            X_mb = M_mb*X_mb + (1-M_mb)*Z_mb
            #H_mb_temp = binary_sampler(self.hint_rate, shape=X_mb.shape)
            #H_mb = M_mb * H_mb_temp
            X_mb[M_mb == 0] = np.nan
            Y_mb[M_mb == 1] = np.nan
            x = np.append(x, [X_mb], axis=0)
            #m = np.append(m, [M_mb], axis=0)
            #h = np.append(h, [H_mb], axis=0)
            y = np.append(y, [Y_mb], axis=0)
            
        #return [x, m, h], y
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

# dgen = GainDataGenerator(df)

In [26]:
class GAIN(keras.Model):
    def __init__(self, shape, alpha=100., load=False, hint_rate=0.9, gen_sigmoid=True, **kwargs):
        super(GAIN, self).__init__(**kwargs)
        self.shape = shape
        self.dim = np.prod(shape).astype(int)
        self.h_dim = self.dim
        self.gen_sigmoid = gen_sigmoid
        self.build_generator()
        self.build_discriminator()
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.generator_optimizer = Adam()
        self.discriminator_optimizer = Adam()

    ## GAIN models
    def build_generator(self):
        last_activation = 'sigmoid' if self.gen_sigmoid else None
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        shape = self.shape
        #x = Input(shape=(self.dim,), name='generator_input_x')
        #m = Input(shape=(self.dim,), name='generator_input_m')
        x = Input(shape=shape, name='generator_input_x')
        m = Input(shape=shape, name='generator_input_m')
        
        x_f = Flatten()(x)
        m_f = Flatten()(m)

        a = Concatenate()([x_f, m_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.dim, activation=last_activation, kernel_initializer=xavier_initializer)(a)
        G_prob = keras.layers.Reshape(shape)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()
        shape = self.shape

        #x = Input(shape=(self.dim,), name='discriminator_input_x')
        #h = Input(shape=(self.dim,), name='discriminator_input_h')
        x = Input(shape=shape, name='discriminator_input_x')
        h = Input(shape=shape, name='discriminator_input_h')
        
        x_f = Flatten()(x)
        h_f = Flatten()(h)

        a = Concatenate()([x_f, h_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        D_prob = keras.layers.Reshape(shape)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        if isinstance(inputs, tuple):
            inputs = inputs[0]
        shape = inputs.shape
        dims = np.prod(shape[1:])
        input_width = shape[1]
        #print('inputs.shape=',inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        #print('dims=',dims)
        #x = keras.layers.Reshape((dims,))(x)
        #x = keras.layers.Reshape(tf.TensorShape((self.dim,)))(x)
        #print('x =', x)
        #print('x.shape = ', x.shape)
        #x = keras.layers.Reshape(tf.TensorShape([57]))(x)
        
        isnan = tf.math.is_nan(x)
        #m = 1.- keras.backend.cast(isnan, dtype=tf.float32)
        m = tf.where(isnan, 0., 1.)
        z = keras.backend.random_uniform(shape=tf.shape(x), minval=0.0, maxval=0.01)
        x = tf.where(isnan, z, x)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        #imputed_data = m*x + (1-m)*imputed_data
        #imputed_data = tf.where(isnan, imputed_data, np.nan)
        imputed_data = tf.where(isnan, imputed_data, x)
        #imputed_data = keras.layers.Reshape(shape[1:])(imputed_data)
        #print('imputed_data.shape = ', imputed_data.shape)
        
        return imputed_data
    
    def D_loss(M, D_prob):
        ## GAIN loss
        return -tf.reduce_mean(M * tf.keras.backend.log(D_prob + 1e-8) \
                         + (1-M) * tf.keras.backend.log(1. - D_prob + 1e-8))
    
    def G_loss(self, M, D_prob, X, G_sample):
        G_loss_temp = -tf.reduce_mean((1-M) * tf.keras.backend.log(D_prob + 1e-8))
        MSE_loss = tf.reduce_mean((M * X - M * G_sample)**2) / (tf.reduce_mean(M) + 1e-8)
        #G_loss_temp = GAIN.G_loss_bincross(M, D_prob)
        #MSE_loss = GAIN.MSE_loss(M, X, G_sample)
        G_loss = G_loss_temp + self.alpha * MSE_loss
        return G_loss
        
    def RMSE_loss(y_true, y_pred):
        isnan = tf.math.is_nan(y_true)
        M = tf.where(isnan, 1., 0.)
        return tf.sqrt(tf.reduce_sum(tf.where(isnan, 0., y_pred-y_true)**2)/tf.reduce_sum(1-M))
    
    def train_step(self, data):
        #[x, m, h], y = data
        x, y = data
        #X = keras.layers.Reshape((self.dim,), input_shape=self.shape)(x)
        #Y = keras.layers.Reshape((self.dim,), input_shape=self.shape)(y)
        #X = keras.layers.Flatten()(x)
        #Y = keras.layers.Flatten()(y)
        X = x
        Y = y
        #X = tf.reshape(x, shape=(x.shape[0], -1))
        #Y = tf.reshape(y, shape=(x.shape[0], -1))
        isnan = tf.math.is_nan(X)
        #M = 1 - keras.backend.cast(isnan, dtype=tf.float32)
        M = tf.where(isnan, 0., 1.)
        Z = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=0.01)
        #H_temp = binary_sampler(self.hint_rate, shape=X.shape)
        H_rand = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=1.)
        #H_temp = 1*keras.backend.cast((H_rand < self.hint_rate), dtype=tf.float32)
        H_temp = tf.where(H_rand < self.hint_rate, 1., 0.)
        
        H = M * H_temp
        #X = M * X + (1-M) * Z
        X = tf.where(isnan, Z, X)
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            G_sample = self.generator([X, M], training=True)

            # Combine with observed data
            #Hat_X = tf.where(isnan, G_sample, X)
            Hat_X = X * M + G_sample * (1-M)
            D_prob = self.discriminator([Hat_X, H], training=True)
            gen_loss = self.G_loss(M, D_prob, X, G_sample)
            disc_loss = tf.keras.backend.mean(tf.keras.losses.binary_crossentropy(M, D_prob))
            #disc_loss = GAIN.D_loss(M, D_prob)
            #disc_loss = GAIN.D_loss(M, D_prob)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        rmse = tf.sqrt(tf.reduce_sum(tf.where(isnan, G_sample - Y, 0.)**2)/tf.reduce_sum(1-M))
        return {
                 'gen_loss':     gen_loss,
                 'disc_loss':    disc_loss,
                 'rmse':         rmse,
               }
    
    def save(self, save_dir='save'):
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        self.discriminator.save_weights(disc_savefile)
        self.generator.save_weights(gen_savefile)

    def load(self, save_dir='save'):
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        try:
          self.discriminator.load_weights(disc_savefile)
          self.generator.load_weights(gen_savefile)
          print('model weights loaded')
        except:
          print('model loadinng error')

In [27]:
def make_dataset_gain(self, data):
  dg = GainDataGenerator(
      df,
      input_width = self.input_width,
      label_width = self.label_width,
      batch_size = 128,
      normalize = False,
      miss_pattern = True,
      miss_rate = 0.15,
      fill_no = 3,
  )
  self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.shape,
        dg.shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_gain

# 측정망별 반복 구간 

In [28]:
norm_df = pd.concat(df,axis=0)
norm_data = norm_df.to_numpy()
MissData.save(norm_data, max_tseq = 24)
# 1: 자동, 2: 수질, 

[[    0     8     2     0]
 [    2     6     1     8]
 [    3     1     1    14]
 ...
 [ 6346    13     3 18669]
 [ 6349    19     4 18682]
 [ 6353     1     1 18701]]
miss_data file saved


In [29]:
dgen = GainDataGenerator(df)

pattern none


In [30]:
train_df = df_all
val_df = df_all
test_df = df_all
df_all

,수온,수소이온농도,전기전도도,용존산소,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,4.7,7.8,333.0,11.1,1.4,NaN,0.004,6.7,-1.407890e-12,1.000000,-0.002666,0.999996
1,4.7,7.8,333.0,11.1,1.3,NaN,0.007,6.8,2.588190e-01,0.965926,-0.001950,0.999998
2,4.7,7.8,333.0,11.0,1.3,NaN,0.007,7.0,5.000000e-01,0.866025,-0.001233,0.999999
3,4.7,7.8,333.0,11.0,1.3,NaN,0.005,7.0,7.071068e-01,0.707107,-0.000516,1.000000
4,4.7,7.8,333.0,11.0,1.4,NaN,0.005,7.1,8.660254e-01,0.500000,0.000201,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
35059,3.9,7.0,240.0,8.2,3.5,1.514,0.027,0.8,-9.659258e-01,0.258819,-0.005734,0.999984
35060,3.9,7.0,240.0,8.2,3.5,1.507,0.027,0.8,-8.660254e-01,0.500000,-0.005017,0.999987
35061,3.9,7.0,240.0,8.2,3.5,1.497,0.026,0.8,-7.071068e-01,0.707107,-0.004301,0.999991
35062,3.8,7.0,240.0,8.2,3.5,1.512,0.025,0.8,-5.000000e-01,0.866025,-0.003584,0.999994


In [31]:
wide_window = WindowGenerator(
    input_width=24*5, label_width=24*5, shift=0,
    #label_columns=['T (degC)']
)

wide_window

Total window size: 120
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
Label indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 

## Graph 

In [32]:
wide_window.example[0].shape

load save


TensorShape([128, 120, 12])

In [33]:
wide_window.plot(plot_col=target_col)
print('make_dataset_gain: dg.no = ', wide_window.dg.no)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

make_dataset_gain: dg.no =  17409


In [34]:
plt.figure(figsize=(9,10))
isnan = np.isnan(norm_data).astype(int)
data = isnan
n = data.shape[0]
seq_len = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(data[i*seq_len:(i+1)*seq_len, 0:], aspect='auto')
    plt.yticks([])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 컴파일 및 학습_GAIN

In [35]:
val_performance = {}
performance = {}

In [36]:
gain = GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False)
gain.compile(loss=GAIN.RMSE_loss)

In [37]:
MAX_EPOCHS = 2000

def compile_and_fit(model, window, patience=10):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  #model.compile(loss=tf.losses.MeanSquaredError(),
                #optimizer=tf.optimizers.Adam(),
                #metrics=[tf.metrics.MeanAbsoluteError()])
  model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

**모델 학습**

In [38]:
history = compile_and_fit(gain, wide_window, patience=MAX_EPOCHS//5)


val_performance['Gain'] = gain.evaluate(wide_window.val)
performance['Gain'] = gain.evaluate(wide_window.test, verbose=0)

# gain.save(save_dir='save')

load save
load save
Epoch 1/2000
1/1 [==============================] - 1s 1s/step - gen_loss: 116.3106 - disc_loss: 0.7242 - rmse: 1.0244 - val_loss: 0.9455
Epoch 2/2000
1/1 [==============================] - 0s 114ms/step - gen_loss: 92.9802 - disc_loss: 0.5292 - rmse: 0.9469 - val_loss: 0.8329
Epoch 3/2000
1/1 [==============================] - 0s 103ms/step - gen_loss: 77.2239 - disc_loss: 0.5021 - rmse: 0.8443 - val_loss: 0.7688
Epoch 4/2000
1/1 [==============================] - 0s 109ms/step - gen_loss: 67.8582 - disc_loss: 0.4043 - rmse: 0.7643 - val_loss: 0.7114
Epoch 5/2000
1/1 [==============================] - 0s 127ms/step - gen_loss: 55.3926 - disc_loss: 0.3921 - rmse: 0.7145 - val_loss: 0.7036
Epoch 6/2000
1/1 [==============================] - 0s 113ms/step - gen_loss: 49.5288 - disc_loss: 0.4021 - rmse: 0.6538 - val_loss: 0.6432
Epoch 7/2000
1/1 [==============================] - 0s 134ms/step - gen_loss: 41.1292 - disc_loss: 0.3885 - rmse: 0.6137 - val_loss: 0.6700
Ep

1/1 [==============================] - 0s 127ms/step - gen_loss: 5.1555 - disc_loss: 0.3107 - rmse: 0.2886 - val_loss: 0.3162
Epoch 60/2000
1/1 [==============================] - 0s 157ms/step - gen_loss: 6.0005 - disc_loss: 0.3068 - rmse: 0.2982 - val_loss: 0.3054
Epoch 61/2000
1/1 [==============================] - 0s 112ms/step - gen_loss: 5.2731 - disc_loss: 0.2947 - rmse: 0.2926 - val_loss: 0.3260
Epoch 62/2000
1/1 [==============================] - 0s 135ms/step - gen_loss: 5.3231 - disc_loss: 0.3003 - rmse: 0.2923 - val_loss: 0.3119
Epoch 63/2000
1/1 [==============================] - 0s 138ms/step - gen_loss: 5.2693 - disc_loss: 0.3041 - rmse: 0.3014 - val_loss: 0.3096
Epoch 64/2000
1/1 [==============================] - 0s 144ms/step - gen_loss: 5.3633 - disc_loss: 0.2911 - rmse: 0.2997 - val_loss: 0.2877
Epoch 65/2000
1/1 [==============================] - 0s 114ms/step - gen_loss: 4.7260 - disc_loss: 0.2927 - rmse: 0.2750 - val_loss: 0.2922
Epoch 66/2000
1/1 [===============

1/1 [==============================] - 0s 146ms/step - gen_loss: 3.3587 - disc_loss: 0.2537 - rmse: 0.2446 - val_loss: 0.2529
Epoch 118/2000
1/1 [==============================] - 0s 142ms/step - gen_loss: 3.7340 - disc_loss: 0.2566 - rmse: 0.2413 - val_loss: 0.2625
Epoch 119/2000
1/1 [==============================] - 0s 140ms/step - gen_loss: 3.4288 - disc_loss: 0.2501 - rmse: 0.2304 - val_loss: 0.2714
Epoch 120/2000
1/1 [==============================] - 0s 142ms/step - gen_loss: 3.8903 - disc_loss: 0.2552 - rmse: 0.2597 - val_loss: 0.2646
Epoch 121/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 3.6734 - disc_loss: 0.2497 - rmse: 0.2453 - val_loss: 0.2702
Epoch 122/2000
1/1 [==============================] - 0s 144ms/step - gen_loss: 3.6169 - disc_loss: 0.2413 - rmse: 0.2553 - val_loss: 0.2551
Epoch 123/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 3.9672 - disc_loss: 0.2451 - rmse: 0.2713 - val_loss: 0.2613
Epoch 124/2000
1/1 [========

1/1 [==============================] - 0s 123ms/step - gen_loss: 3.3616 - disc_loss: 0.2177 - rmse: 0.2556 - val_loss: 0.2631
Epoch 176/2000
1/1 [==============================] - 0s 113ms/step - gen_loss: 3.0621 - disc_loss: 0.2104 - rmse: 0.2284 - val_loss: 0.2440
Epoch 177/2000
1/1 [==============================] - 0s 127ms/step - gen_loss: 3.4752 - disc_loss: 0.2143 - rmse: 0.2552 - val_loss: 0.2404
Epoch 178/2000
1/1 [==============================] - 0s 144ms/step - gen_loss: 3.1103 - disc_loss: 0.2162 - rmse: 0.2454 - val_loss: 0.2488
Epoch 179/2000
1/1 [==============================] - 0s 161ms/step - gen_loss: 3.5386 - disc_loss: 0.2250 - rmse: 0.2666 - val_loss: 0.2445
Epoch 180/2000
1/1 [==============================] - 0s 130ms/step - gen_loss: 3.4702 - disc_loss: 0.2130 - rmse: 0.2575 - val_loss: 0.2801
Epoch 181/2000
1/1 [==============================] - 0s 124ms/step - gen_loss: 3.1742 - disc_loss: 0.2230 - rmse: 0.2471 - val_loss: 0.2418
Epoch 182/2000
1/1 [========

1/1 [==============================] - 0s 117ms/step - gen_loss: 2.8704 - disc_loss: 0.1939 - rmse: 0.2462 - val_loss: 0.2406
Epoch 234/2000
1/1 [==============================] - 0s 120ms/step - gen_loss: 2.7294 - disc_loss: 0.1969 - rmse: 0.2147 - val_loss: 0.2747
Epoch 235/2000
1/1 [==============================] - 0s 130ms/step - gen_loss: 2.8144 - disc_loss: 0.2011 - rmse: 0.2417 - val_loss: 0.2675
Epoch 236/2000
1/1 [==============================] - 0s 128ms/step - gen_loss: 2.9113 - disc_loss: 0.1989 - rmse: 0.2362 - val_loss: 0.2394
Epoch 237/2000
1/1 [==============================] - 0s 136ms/step - gen_loss: 2.7590 - disc_loss: 0.1968 - rmse: 0.2248 - val_loss: 0.2319
Epoch 238/2000
1/1 [==============================] - 0s 128ms/step - gen_loss: 2.5871 - disc_loss: 0.1942 - rmse: 0.2350 - val_loss: 0.2492
Epoch 239/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 2.6149 - disc_loss: 0.1937 - rmse: 0.2175 - val_loss: 0.2393
Epoch 240/2000
1/1 [========

...

Epoch 1332/2000
1/1 [==============================] - 0s 123ms/step - gen_loss: 1.7674 - disc_loss: 0.1267 - rmse: 0.2278 - val_loss: 0.2232
Epoch 1333/2000
1/1 [==============================] - 0s 124ms/step - gen_loss: 1.6737 - disc_loss: 0.1270 - rmse: 0.1981 - val_loss: 0.2255
Epoch 1334/2000
1/1 [==============================] - 0s 134ms/step - gen_loss: 1.7928 - disc_loss: 0.1328 - rmse: 0.2616 - val_loss: 0.1977
Epoch 1335/2000
1/1 [==============================] - 0s 105ms/step - gen_loss: 1.7346 - disc_loss: 0.1265 - rmse: 0.2317 - val_loss: 0.2200
Epoch 1336/2000
1/1 [==============================] - 0s 134ms/step - gen_loss: 1.7340 - disc_loss: 0.1267 - rmse: 0.2450 - val_loss: 0.2200
Epoch 1337/2000
1/1 [==============================] - 0s 139ms/step - gen_loss: 1.7876 - disc_loss: 0.1269 - rmse: 0.2179 - val_loss: 0.2149
Epoch 1338/2000
1/1 [==============================] - 0s 124ms/step - gen_loss: 1.7861 - disc_loss: 0.1292 - rmse: 0.2135 - val_loss: 0.2297
Epoch 

1/1 [==============================] - 0s 142ms/step - gen_loss: 1.8428 - disc_loss: 0.1338 - rmse: 0.1949 - val_loss: 0.2250
Epoch 1390/2000
1/1 [==============================] - 0s 140ms/step - gen_loss: 2.0387 - disc_loss: 0.1329 - rmse: 0.2385 - val_loss: 0.2518
Epoch 1391/2000
1/1 [==============================] - 0s 126ms/step - gen_loss: 1.9912 - disc_loss: 0.1301 - rmse: 0.2166 - val_loss: 0.2493
Epoch 1392/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 1.7404 - disc_loss: 0.1319 - rmse: 0.2048 - val_loss: 0.2178
Epoch 1393/2000
1/1 [==============================] - 0s 130ms/step - gen_loss: 1.7443 - disc_loss: 0.1308 - rmse: 0.2034 - val_loss: 0.2369
Epoch 1394/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 2.1120 - disc_loss: 0.1306 - rmse: 0.2235 - val_loss: 0.2309
Epoch 1395/2000
1/1 [==============================] - 0s 120ms/step - gen_loss: 2.1334 - disc_loss: 0.1347 - rmse: 0.2252 - val_loss: 0.2698
Epoch 1396/2000
1/1 [=

1/1 [==============================] - 0s 120ms/step - gen_loss: 1.8050 - disc_loss: 0.1306 - rmse: 0.2230 - val_loss: 0.2334
Epoch 1447/2000
1/1 [==============================] - 0s 115ms/step - gen_loss: 1.8173 - disc_loss: 0.1312 - rmse: 0.2223 - val_loss: 0.2382
Epoch 1448/2000
1/1 [==============================] - 0s 105ms/step - gen_loss: 1.7865 - disc_loss: 0.1320 - rmse: 0.2009 - val_loss: 0.2363
Epoch 1449/2000
1/1 [==============================] - 0s 102ms/step - gen_loss: 1.8440 - disc_loss: 0.1285 - rmse: 0.2089 - val_loss: 0.2421
Epoch 1450/2000
1/1 [==============================] - 0s 126ms/step - gen_loss: 1.9138 - disc_loss: 0.1285 - rmse: 0.2255 - val_loss: 0.2290
Epoch 1451/2000
1/1 [==============================] - 0s 107ms/step - gen_loss: 1.7798 - disc_loss: 0.1266 - rmse: 0.2252 - val_loss: 0.2506
Epoch 1452/2000
1/1 [==============================] - 0s 122ms/step - gen_loss: 1.8428 - disc_loss: 0.1295 - rmse: 0.1983 - val_loss: 0.2214
Epoch 1453/2000
1/1 [=

1/1 [==============================] - 0s 130ms/step - gen_loss: 2.0186 - disc_loss: 0.1287 - rmse: 0.2351 - val_loss: 0.2200
Epoch 1504/2000
1/1 [==============================] - 0s 145ms/step - gen_loss: 2.1176 - disc_loss: 0.1235 - rmse: 0.2272 - val_loss: 0.2315
Epoch 1505/2000
1/1 [==============================] - 0s 133ms/step - gen_loss: 1.9215 - disc_loss: 0.1291 - rmse: 0.2283 - val_loss: 0.2601
Epoch 1506/2000
1/1 [==============================] - 0s 135ms/step - gen_loss: 1.9445 - disc_loss: 0.1270 - rmse: 0.2322 - val_loss: 0.2185
Epoch 1507/2000
1/1 [==============================] - 0s 125ms/step - gen_loss: 1.8524 - disc_loss: 0.1259 - rmse: 0.2267 - val_loss: 0.2110
Epoch 1508/2000
1/1 [==============================] - 0s 91ms/step - gen_loss: 1.9474 - disc_loss: 0.1230 - rmse: 0.2302 - val_loss: 0.2150
Epoch 1509/2000
1/1 [==============================] - 0s 134ms/step - gen_loss: 1.7971 - disc_loss: 0.1250 - rmse: 0.2394 - val_loss: 0.2563
Epoch 1510/2000
1/1 [==

Epoch 1561/2000
1/1 [==============================] - 0s 142ms/step - gen_loss: 1.7345 - disc_loss: 0.1210 - rmse: 0.2048 - val_loss: 0.2219
Epoch 1562/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 1.7510 - disc_loss: 0.1278 - rmse: 0.2328 - val_loss: 0.2390
Epoch 1563/2000
1/1 [==============================] - 0s 124ms/step - gen_loss: 1.6688 - disc_loss: 0.1243 - rmse: 0.2109 - val_loss: 0.2182
Epoch 1564/2000
1/1 [==============================] - 0s 128ms/step - gen_loss: 1.6616 - disc_loss: 0.1219 - rmse: 0.2179 - val_loss: 0.2318
Epoch 1565/2000
1/1 [==============================] - 0s 113ms/step - gen_loss: 1.7436 - disc_loss: 0.1226 - rmse: 0.2286 - val_loss: 0.2697
Epoch 1566/2000
1/1 [==============================] - 0s 123ms/step - gen_loss: 1.6667 - disc_loss: 0.1267 - rmse: 0.1931 - val_loss: 0.2553
Epoch 1567/2000
1/1 [==============================] - 0s 107ms/step - gen_loss: 1.7176 - disc_loss: 0.1279 - rmse: 0.2183 - val_loss: 0.2634
Epoch 

**학습 loss history 출력**

In [39]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(history.history['gen_loss'], label='gen_loss')
ax.plot(history.history['disc_loss'], label='disc_loss')
ax2.plot(history.history['rmse'], label='rmse', color='green')
ax2.plot(history.history['val_loss'], label='val_loss', color='red')
#plt.legend(history.history.keys(), loc='upper right')
#ax.legend(loc='upper center')
ax.legend(loc='upper center')
ax2.legend(loc='upper right')
ax.set_xlabel("epochs")
ax.set_ylabel("loss")
ax2.set_ylabel("rmse")
plt.show()    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**성능 측정**

In [40]:
gain.evaluate(wide_window.test.repeat(), steps=100)

load save
100/100 [==============================] - 3s 29ms/step - loss: 0.2196


0.21963676810264587

**model save**

In [41]:
gain.save(save_dir='save')

**샘플 prediction 출력**

In [42]:
wide_window.plot(gain, plot_col='총유기탄소')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
norm_df = pd.concat(df,axis=0)

In [44]:
data = norm_df.to_numpy()

total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print(unit_shape)
#dim = np.prod(wide_window.dg.shape[1:]).astype(int)
dim = wide_window.dg.shape[1]
print(dim)
n = (total_n//dim)*dim

x = data[0:n].copy()
y_true = data[0:n].copy()

#x = interpolate(x, max_gap=3)

print('x.shape =', x.shape)
x_reshape = x.reshape((-1,)+unit_shape)
print('x_reshape.shape =', x_reshape.shape)
isnan = np.isnan(x_reshape)
print(isnan.sum())
print('y_true.shape=', y_true.shape)
isnan = np.isnan(y_true)
print(isnan.sum())

x_remain = data[-wide_window.dg.shape[1]:].copy()
x_remain_reshape = x_remain.reshape((-1,)+unit_shape)
x_remain_reshape.shape

105192
(120, 12)
120
x.shape = (105120, 12)
x_reshape.shape = (876, 120, 12)
92854
y_true.shape= (105120, 12)
92854


(1, 120, 12)

In [45]:
gain.evaluate(x_reshape, y_true.reshape((-1,)+unit_shape))

28/28 [==============================] - 0s 2ms/step - loss: 0.0000e+00


0.0

In [46]:
y_pred = gain.predict(x_reshape)
y_remain_pred = gain.predict(x_remain_reshape)

In [47]:
y_pred = y_pred.reshape(y_true.shape)
y_remain_pred = y_remain_pred.reshape(x_remain.shape)
print(y_pred.shape, y_remain_pred.shape)
y_pred = np.append(y_pred, y_remain_pred[-(total_n-n):], axis=0)
print(y_pred.shape)

(105120, 12) (120, 12)
(105192, 12)


In [48]:
y_pred[~np.isnan(data)] = np.nan

In [49]:
n = 8
plt.figure(figsize=(9,20))
for i in range(n):
    #plt.subplot('%d1%d'%(n,i))
    plt.subplot(811+i)
    plt.plot(x[:, i])
    plt.plot(y_pred[:, i])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Imputed Data**

In [50]:
total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print('unit_shape=', unit_shape)
time_seq = unit_shape[0]
print(time_seq)
n = (total_n//time_seq)*time_seq
print('n=', n)

gans = []
oris = []
for i in range(len(df)):
    x = df[i].to_numpy()
    total_n = x.shape[0]
    n = (total_n//time_seq)*time_seq
    x = x[0:n]
    x_block = x.reshape((-1,)+unit_shape)
    y = gain.predict(x_block)
    y_gan = y.reshape(x.shape)
    
    # cut off sin, cos data
    if (i > 0):
        x = x[:, :-4]
        y_gan = y_gan[:, :-4]
    gans.append(y_gan)
    oris.append(x)
print(x.shape)
print(y_gan.shape)

105192
unit_shape= (120, 12)
120
n= 105120
(35040, 8)
(35040, 8)


# 예측

In [51]:
df[0].head

<bound method NDFrame.head of              수온    수소이온농도     전기전도도      용존산소     총유기탄소       총질소        총인  \
0     -1.247648 -0.265085  2.187300  0.329570 -1.749304       NaN -0.670034   
1     -1.247648 -0.265085  2.187300  0.329570 -1.862182       NaN -0.538826   
2     -1.247648 -0.265085  2.187300  0.288852 -1.862182       NaN -0.538826   
3     -1.247648 -0.265085  2.187300  0.288852 -1.862182       NaN -0.626298   
4     -1.247648 -0.265085  2.187300  0.288852 -1.749304       NaN -0.626298   
...         ...       ...       ...       ...       ...       ...       ...   
35059 -1.345284  0.234418  0.696041 -0.077605 -0.846285  0.781209 -0.582562   
35060 -1.345284  0.234418  0.650852 -0.077605 -0.733407  0.860842 -0.626298   
35061 -1.345284  0.234418  0.605662 -0.118322 -0.733407  0.851994 -0.582562   
35062 -1.366980  0.234418  0.628257 -0.118322 -0.846285  0.844410 -0.451354   
35063 -1.366980  0.234418  0.628257 -0.118322 -0.846285  0.800169 -0.626298   

         클로로필-a      

In [52]:
ori = np.concatenate(oris, axis=1)
gan = np.concatenate(gans, axis=1)
# print(oris[0].shape, gans[0].shape)
# print(oris[1].shape, gans[1].shape)
# print(ori.shape, gan.shape)

total_no = ori.shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gan[train_slice])
val_df = pd.DataFrame(gan[val_slice])
test_df = pd.DataFrame(gan[test_slice])

train_ori_df = pd.DataFrame(ori[train_slice])
val_ori_df = pd.DataFrame(ori[val_slice])
test_ori_df = pd.DataFrame(ori[test_slice])

num_features = train_df.shape[1]
#out_num_features = oris[0].shape[1]-4
# out_features = [target_col_idx]
# out_num_features = len(out_features)
# out_num_features

In [53]:
train_df 

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,-1.247648,-0.265085,2.187300,0.329570,-1.749304,-0.079002,-0.670034,-0.792973,-1.999586e-12,1.414207,...,-0.276410,-1.040758,-1.377829,0.567419,1.690214,1.306788,-0.620530,0.437396,-0.407618,-0.586485
1,-1.247648,-0.265085,2.187300,0.329570,-1.862182,-0.120712,-0.538826,-0.786090,3.660237e-01,1.366019,...,-0.451354,-1.033876,-1.377829,0.567419,1.690214,1.306788,-0.620530,0.479109,-0.495090,-0.572719
2,-1.247648,-0.265085,2.187300,0.288852,-1.862182,-0.074645,-0.538826,-0.772324,7.071034e-01,1.224739,...,-0.451354,-1.033876,-1.388677,0.567419,1.712809,1.306788,-0.620530,0.500597,-0.363882,-0.586485
3,-1.247648,-0.265085,2.187300,0.288852,-1.862182,-0.055247,-0.626298,-0.772324,9.999952e-01,0.999995,...,-0.101466,-1.026993,-1.377829,0.567419,1.712809,1.266071,-0.620530,0.475317,-0.451354,-0.565836
4,-1.247648,-0.265085,2.187300,0.288852,-1.749304,-0.105983,-0.626298,-0.765441,1.224739e+00,0.707103,...,-0.276410,-1.013227,-1.388677,0.567419,1.712809,1.225353,-0.620530,0.494277,-0.538826,-0.565836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24523,0.042653,-0.370604,-0.415594,-0.210324,0.395368,0.997356,0.729517,-0.579602,-1.366019e+00,0.366024,...,-0.013994,-0.565836,0.010768,0.733920,-0.456295,0.981049,0.395368,0.413380,0.248421,1.746832
24524,-0.014635,-0.434593,-0.325809,-0.024171,0.395368,1.127550,0.860725,-0.614016,-1.224739e+00,0.707103,...,0.029741,-0.586485,0.010768,0.733920,-0.456295,0.940331,0.395368,0.342595,0.204685,1.753715
24525,-0.122678,-0.308782,-0.381074,-0.095284,0.395368,1.047917,0.685781,-0.648431,-9.999952e-01,0.999995,...,0.117213,-0.497006,0.010768,0.733920,-0.433701,0.940331,0.395368,0.313522,0.248421,1.788129
24526,0.117861,-0.201225,-0.275065,-0.073440,0.395368,1.126286,0.816989,-0.655314,-7.071034e-01,1.224739,...,0.029741,-0.469475,-0.000081,0.567419,-0.433701,0.858896,0.395368,0.297090,0.248421,1.643588


In [54]:
df[0].columns

Index(['수온', '수소이온농도', '전기전도도', '용존산소', '총유기탄소', '총질소', '총인', '클로로필-a',
       'Day sin', 'Day cos', 'Year sin', 'Year cos'],
      dtype='object')

In [221]:
MAX_EPOCHS = 150
target_col = '클로로필-a'
num_features = df[0].shape[1]
label_columns_indices = {name: i for i, name in enumerate(df[0])}
target_col_idx = label_columns_indices[target_col]
# target_col_idx
out_features = [target_col_idx]
# out_features = [3,4,5,6,7]
out_num_features = len(out_features)
out_num_features

1

In [222]:
out_features

[7]

In [223]:
class WaterDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size=32,
                 input_width=24*7,
                 label_width=24*3,
                 shift=24*3,
                 skip_time = None,
                 shuffle = True,
                 out_features = None,
                 out_num_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
#         print("out_features")
#         print(out_features)
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        x = np.empty((0, self.input_width, self.data.shape[1]))
        
        #y = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.label_width, self.out_num_features))
       
        #print(x.shape)
        #print(y.shape)
        
        
        for cnt in range(0, self.batch_size):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2].to_numpy()
            
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features].to_numpy()
            #Y = self.data[idx1:idx2]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            
            self.batch_id %= self.no
            #print("x.shape=", x.shape)
            #print('X.shape=', X.shape)
            #print(type(x), type(X))
            x = np.append(x, [X], axis = 0)
            
            y = np.append(y, [Y], axis = 0)
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

**WaterDataGenerator 테스트**

In [224]:
OUT_STEPS = 24*5

In [225]:
# OUT_STEPS = 24*3
# wdg = WaterDataGenerator(train_df,
#                          batch_size=128,
#                          input_width = 24*7,
#                          label_width = OUT_STEPS,
#                          shift = OUT_STEPS,
#                          out_num_features = out_num_features,
# #                          out_features = out_features
#                         )

In [226]:
# train_df

In [227]:
# it = iter(wdg)

In [228]:
# x,y = next(it)
# x.shape, y.shape

In [229]:
def make_dataset_water(self, data):
#   print("make_dataset_water")
#   print(out_features)
  dg = WaterDataGenerator(
      data,
      batch_size = 128,
      input_width = self.input_width,
      label_width = self.label_width,
      shift = self.label_width,
      out_features = out_features,
      out_num_features = out_num_features,
  )
  #self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.input_shape,
        dg.label_shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_water

In [230]:
def plot2(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
      label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
      label_col_index = plot_col_index
      label_out_col_index = plot_out_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_out_col_index],
                label='Labels', c='#2ca02c')
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_out_col_index],
                  marker=None, label='Predictions',
                  c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot2 = plot2

**1일단위로 변환**

In [231]:
def hour_to_day_mean(array):
    
#     print(array)
    
    time = 24
    array = tf.reduce_mean(tf.reshape(array, [array.shape[0]//time, time]), 1)
#     print(array)
    
    return array

def plot24(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
    
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))

  if self.label_columns:
    label_col_index = self.label_columns_indices.get(plot_col, None)
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_col_index = plot_col_index
    label_out_col_index = plot_out_col_index
    
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{df_all.columns[plot_col]} [normed]', fontproperties=fprop)
    
    input_temp = hour_to_day_mean(inputs[n, :, plot_col_index])
    input_temp = (input_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#     input_temp = input_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.input_indices),
        input_temp,
        label='Inputs', marker='.', zorder=-10)

    if label_col_index is None:
      continue
    
    label_temp = hour_to_day_mean(labels[n, :, label_out_col_index])
    label_temp = (label_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#     label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.label_indices),
        label_temp,
        label='Labels', marker='.', zorder=-10, c='#2ca02c')
    
    if model is not None:
      predictions = model(inputs)
    
      #predictions = predictions * train_std[plot_col] * train_mean[plot_col]
      predict_temp = hour_to_day_mean(predictions[n,:, label_out_col_index])
      predict_temp = (predict_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#       predict_temp = predict_temp * train_std[plot_col] + train_mean[plot_col]
    
      plt.plot(
          hour_to_day_mean(self.label_indices),
          predict_temp,
          label='Predictions', marker='.', zorder=-10, c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [day]')

WindowGenerator.plot24 = plot24

**예측률 계산**

In [232]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [233]:
def compa(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0, windows=None, min_max_normailze=False):
       
    if example is not None :
        inputs, labels = windows
    else :
        inputs, labels = self.example
        
        
    plot_out_col_index = self.column_indices[plot_out_col]
    
    if self.label_columns:
        label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
        label_out_col_index = plot_out_col_index
    
    if model is None:
        return
    
    print("column : " + df_all.columns[plot_col])
    
    mae = 0
    mse = 0
    rmse = 0
    mape = 0
    pred_arr = []
    label_arr = []
    o = 0
    o1 = 0
    p = 0
    nse_sum1 = 0 
    nse_sum2 = 0
    pbias_sum1 = 0
    pbias_sum2 = 0
    
    
    predictions = model(inputs)
    

    
    predictions = predictions * train_std[plot_col] + train_mean[plot_col]
    labels = labels * train_std[plot_col] + train_mean[plot_col]
    
    
#     print(predictions)
    
#     print(labels)
    o1 = np.mean(labels.numpy())
#     print('o1')
#     print(o1)
    
    if min_max_normailze :
#         predictions = predictions * train_std[plot_col] + train_mean[plot_col]
#         labels = labels * train_std[plot_col] + train_mean[plot_col]
        
        predictions = predictions.numpy()
        labels = labels.numpy()
        
        predictions = (predictions - predictions.min()) / (predictions.max() - predictions.min())
        labels = (labels - labels.min()) / (labels.max() - labels.min())

    for n in range(len(inputs)):
        pred_temp = hour_to_day_mean(predictions[n,:,label_out_col_index]) 
        label_temp = hour_to_day_mean(labels[n,:,label_out_col_index])
        
        o = label_temp[4].numpy()
        p = pred_temp[4].numpy()
        
#         print('pred_temp')
#         print(pred_temp[4])
#         print('+++++++++++++')
#         print(label_temp[4])
        
#         print('p')
#         print(p)
#         print('o')
#         print(o)
        
        temp_m = o-p
#         print('temp')
#         print(temp_m)
#         print(o)

#         print(o-o1)
#         print('22222')
        
        nse_sum1 = nse_sum1 + temp_m**2
        nse_sum2 = nse_sum2 + (o-o1)**2
        
    
            
        pbias_sum1 = pbias_sum1 + temp_m
        pbias_sum2 = pbias_sum2 + o
            
#         pred_arr.append(pred_temp[4])
#         label_arr.append(label_temp[4])
    
    
#     print(nse_sum1)
#     print('111111')
#     print(nse_sum2)

#     print('nse_sum1')
#     print(nse_sum1)
#     print('nse_sum2')
#     print(nse_sum2)
#     print('pbias_sum1')
#     print(pbias_sum1)
#     print('pbias_sum2')
#     print(pbias_sum2)
    
        
    nse = 1 - (nse_sum1 / nse_sum2)
    pbias = (pbias_sum1 / pbias_sum2) * 100
    
    
    
    
    print('NSE')
    print(nse)
    
    print('PBIAS')
    print(pbias)
    
        
#     mae = mean_absolute_error(label_arr, pred_arr)
#     mse = mean_squared_error(label_arr, pred_arr)
#     rmse = np.sqrt(mse)
#     mape = np.mean(np.abs((np.array(label_arr) - np.array(pred_arr))/np.array(label_arr)) )*100
    
#     print("mae:")  
#     print(mae)
    
#     print("mse:")  
#     print(mse)
    
#     print("rmse")  
#     print(rmse)
    
#     print("mape")  
#     print(mape)
    
    

WindowGenerator.compa = compa

In [234]:
# np.sqrt(-0.28892756)

In [235]:
train_df.shape

(24528, 28)

In [236]:
#multi_window.compa(elman_model, plot_col=out_features[0])

In [237]:
# target_col = '용존산소'
# num_features = df[0].shape[1]
# label_columns_indices = {name: i for i, name in enumerate(df[0])}
# target_col_idx = label_columns_indices[target_col]
# # target_col_idx
# out_features = [target_col_idx]
# # out_features = [3,5,6,7,8]
# out_num_features = len(out_features)
# out_num_features

In [238]:
def compile_and_fit(model, window, patience=1000):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
#   model.compile(loss=tf.losses.MeanSquaredError(),
#                 optimizer=tf.optimizers.Adam(),
#                 metrics=[tf.metrics.MeanSquaredError()])
  #model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [239]:
def plot_history(history):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(history.history['loss'], label='loss')
    ax.plot(history.history['mean_absolute_error'], label='mae')
#     ax.plot(history.history['mean_squared_error'], label='mse')
    ax.plot(history.history['val_loss'], label='val_loss')
    ax.plot(history.history['val_mean_absolute_error'], label='val_mae')
#     ax.plot(history.history['val_mean_squared_error'], label='val_mse')
    #plt.legend(history.history.keys(), loc='upper right')
    #ax.legend(loc='upper center')
    ax.legend()
    ax.set_xlabel("epochs")
    ax.set_ylabel("loss")
    plt.show()

In [240]:
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               out_features = out_features,
                               out_num_features = out_num_features,
                               )

In [241]:
multi_val_performance = {}
multi_performance = {}

**Elman_RNN**

In [242]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [243]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [244]:
history = compile_and_fit(elman_model, multi_window)
multi_val_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['ELMAN_RNN'])
print('test performance = ', multi_performance['ELMAN_RNN'])

Epoch 1/150
1/1 [==============================] - 1s 1s/step - loss: 1.0889 - mean_absolute_error: 0.7386 - val_loss: 0.7374 - val_mean_absolute_error: 0.6393
Epoch 2/150
1/1 [==============================] - 0s 306ms/step - loss: 0.9144 - mean_absolute_error: 0.7226 - val_loss: 0.7196 - val_mean_absolute_error: 0.6329
Epoch 3/150
1/1 [==============================] - 0s 259ms/step - loss: 1.1925 - mean_absolute_error: 0.7742 - val_loss: 0.6754 - val_mean_absolute_error: 0.6039
Epoch 4/150
1/1 [==============================] - 0s 275ms/step - loss: 0.6083 - mean_absolute_error: 0.6308 - val_loss: 0.7077 - val_mean_absolute_error: 0.5805
Epoch 5/150
1/1 [==============================] - 0s 254ms/step - loss: 0.6350 - mean_absolute_error: 0.6354 - val_loss: 0.8267 - val_mean_absolute_error: 0.6864
Epoch 6/150
1/1 [==============================] - 0s 243ms/step - loss: 0.7355 - mean_absolute_error: 0.6154 - val_loss: 0.6054 - val_mean_absolute_error: 0.5600
Epoch 7/150
1/1 [========

1/1 [==============================] - 0s 285ms/step - loss: 0.5530 - mean_absolute_error: 0.4367 - val_loss: 0.4435 - val_mean_absolute_error: 0.4944
Epoch 52/150
1/1 [==============================] - 0s 295ms/step - loss: 0.4828 - mean_absolute_error: 0.4217 - val_loss: 0.3562 - val_mean_absolute_error: 0.4550
Epoch 53/150
1/1 [==============================] - 0s 275ms/step - loss: 0.4036 - mean_absolute_error: 0.3816 - val_loss: 0.5516 - val_mean_absolute_error: 0.5536
Epoch 54/150
1/1 [==============================] - 0s 275ms/step - loss: 0.3647 - mean_absolute_error: 0.4128 - val_loss: 0.5818 - val_mean_absolute_error: 0.5527
Epoch 55/150
1/1 [==============================] - 0s 265ms/step - loss: 0.3013 - mean_absolute_error: 0.3988 - val_loss: 0.4275 - val_mean_absolute_error: 0.4766
Epoch 56/150
1/1 [==============================] - 0s 284ms/step - loss: 0.2490 - mean_absolute_error: 0.3632 - val_loss: 0.4124 - val_mean_absolute_error: 0.4786
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 292ms/step - loss: 0.2519 - mean_absolute_error: 0.3213 - val_loss: 0.4514 - val_mean_absolute_error: 0.5063
Epoch 102/150
1/1 [==============================] - 0s 286ms/step - loss: 0.2664 - mean_absolute_error: 0.3473 - val_loss: 0.4212 - val_mean_absolute_error: 0.4823
Epoch 103/150
1/1 [==============================] - 0s 298ms/step - loss: 0.2991 - mean_absolute_error: 0.3614 - val_loss: 0.4751 - val_mean_absolute_error: 0.5056
Epoch 104/150
1/1 [==============================] - 0s 316ms/step - loss: 0.2719 - mean_absolute_error: 0.3396 - val_loss: 0.4222 - val_mean_absolute_error: 0.4776
Epoch 105/150
1/1 [==============================] - 0s 298ms/step - loss: 0.2380 - mean_absolute_error: 0.3311 - val_loss: 0.5198 - val_mean_absolute_error: 0.5175
Epoch 106/150
1/1 [==============================] - 0s 298ms/step - loss: 0.4966 - mean_absolute_error: 0.3862 - val_loss: 0.4124 - val_mean_absolute_error: 0.4634
Epoch 107/150
1/1 [=====

100/100 [==============================] - 9s 91ms/step - loss: 0.3763 - mean_absolute_error: 0.4053
val performance = [0.629238486289978, 0.5904722213745117]
test performance =  [0.3762575387954712, 0.40527787804603577]


In [245]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [246]:
multi_window.plot2(elman_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**GRU**

In [247]:
gru_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=False),
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [248]:
history = compile_and_fit(gru_model, multi_window)
multi_val_performance['GRU'] = gru_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['GRU'] = gru_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

Epoch 1/150
1/1 [==============================] - 1s 1s/step - loss: 0.9677 - mean_absolute_error: 0.7312 - val_loss: 0.7279 - val_mean_absolute_error: 0.6249
Epoch 2/150
1/1 [==============================] - 0s 313ms/step - loss: 0.9493 - mean_absolute_error: 0.7194 - val_loss: 0.8656 - val_mean_absolute_error: 0.7063
Epoch 3/150
1/1 [==============================] - 0s 338ms/step - loss: 1.2019 - mean_absolute_error: 0.7398 - val_loss: 0.7895 - val_mean_absolute_error: 0.6769
Epoch 4/150
1/1 [==============================] - 0s 337ms/step - loss: 0.6947 - mean_absolute_error: 0.6520 - val_loss: 0.6592 - val_mean_absolute_error: 0.5915
Epoch 5/150
1/1 [==============================] - 0s 350ms/step - loss: 0.9031 - mean_absolute_error: 0.7229 - val_loss: 0.7859 - val_mean_absolute_error: 0.6504
Epoch 6/150
1/1 [==============================] - 0s 351ms/step - loss: 0.8140 - mean_absolute_error: 0.6570 - val_loss: 0.7398 - val_mean_absolute_error: 0.6547
Epoch 7/150
1/1 [========

1/1 [==============================] - 0s 287ms/step - loss: 0.3853 - mean_absolute_error: 0.3832 - val_loss: 0.4574 - val_mean_absolute_error: 0.4746
Epoch 52/150
1/1 [==============================] - 0s 329ms/step - loss: 0.6043 - mean_absolute_error: 0.4512 - val_loss: 0.3796 - val_mean_absolute_error: 0.4493
Epoch 53/150
1/1 [==============================] - 0s 306ms/step - loss: 0.3560 - mean_absolute_error: 0.4014 - val_loss: 0.4784 - val_mean_absolute_error: 0.4777
Epoch 54/150
1/1 [==============================] - 0s 275ms/step - loss: 0.2959 - mean_absolute_error: 0.3803 - val_loss: 0.4053 - val_mean_absolute_error: 0.4327
Epoch 55/150
1/1 [==============================] - 0s 280ms/step - loss: 0.3528 - mean_absolute_error: 0.4081 - val_loss: 0.4203 - val_mean_absolute_error: 0.4476
Epoch 56/150
1/1 [==============================] - 0s 317ms/step - loss: 0.2919 - mean_absolute_error: 0.3929 - val_loss: 0.3889 - val_mean_absolute_error: 0.4290
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 312ms/step - loss: 0.3905 - mean_absolute_error: 0.3743 - val_loss: 0.4335 - val_mean_absolute_error: 0.4555
Epoch 102/150
1/1 [==============================] - 0s 341ms/step - loss: 0.3174 - mean_absolute_error: 0.3262 - val_loss: 0.4174 - val_mean_absolute_error: 0.4491
Epoch 103/150
1/1 [==============================] - 0s 334ms/step - loss: 0.2643 - mean_absolute_error: 0.3377 - val_loss: 0.4867 - val_mean_absolute_error: 0.4914
Epoch 104/150
1/1 [==============================] - 0s 314ms/step - loss: 0.4003 - mean_absolute_error: 0.3771 - val_loss: 0.3800 - val_mean_absolute_error: 0.4506
Epoch 105/150
1/1 [==============================] - 0s 333ms/step - loss: 0.3148 - mean_absolute_error: 0.3789 - val_loss: 0.4268 - val_mean_absolute_error: 0.4515
Epoch 106/150
1/1 [==============================] - 0s 300ms/step - loss: 0.4084 - mean_absolute_error: 0.3744 - val_loss: 0.4504 - val_mean_absolute_error: 0.4833
Epoch 107/150
1/1 [=====

100/100 [==============================] - 10s 102ms/step - loss: 0.3744 - mean_absolute_error: 0.4054
val performance = [0.5653747320175171, 0.5328444838523865]
test performance =  [0.3743760287761688, 0.4054161310195923]


**LSTM**

In [249]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [250]:
history = compile_and_fit(multi_lstm_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['LSTM'])
print('test performance = ', multi_performance['LSTM'])

Epoch 1/150
1/1 [==============================] - 1s 1s/step - loss: 0.6397 - mean_absolute_error: 0.6528 - val_loss: 0.9131 - val_mean_absolute_error: 0.7216
Epoch 2/150
1/1 [==============================] - 0s 257ms/step - loss: 1.1491 - mean_absolute_error: 0.7440 - val_loss: 0.7070 - val_mean_absolute_error: 0.6582
Epoch 3/150
1/1 [==============================] - 0s 276ms/step - loss: 0.8728 - mean_absolute_error: 0.7099 - val_loss: 0.7495 - val_mean_absolute_error: 0.6654
Epoch 4/150
1/1 [==============================] - 0s 258ms/step - loss: 0.7643 - mean_absolute_error: 0.6936 - val_loss: 0.7955 - val_mean_absolute_error: 0.6414
Epoch 5/150
1/1 [==============================] - 0s 229ms/step - loss: 0.7732 - mean_absolute_error: 0.6724 - val_loss: 0.7046 - val_mean_absolute_error: 0.6175
Epoch 6/150
1/1 [==============================] - 0s 256ms/step - loss: 0.8809 - mean_absolute_error: 0.6971 - val_loss: 0.7561 - val_mean_absolute_error: 0.6431
Epoch 7/150
1/1 [========

1/1 [==============================] - 0s 230ms/step - loss: 0.3623 - mean_absolute_error: 0.3680 - val_loss: 0.4627 - val_mean_absolute_error: 0.4507
Epoch 52/150
1/1 [==============================] - 0s 227ms/step - loss: 0.2478 - mean_absolute_error: 0.3647 - val_loss: 0.4440 - val_mean_absolute_error: 0.4693
Epoch 53/150
1/1 [==============================] - 0s 256ms/step - loss: 0.3376 - mean_absolute_error: 0.3713 - val_loss: 0.3782 - val_mean_absolute_error: 0.4282
Epoch 54/150
1/1 [==============================] - 0s 270ms/step - loss: 0.2538 - mean_absolute_error: 0.3348 - val_loss: 0.4838 - val_mean_absolute_error: 0.4749
Epoch 55/150
1/1 [==============================] - 0s 243ms/step - loss: 0.3193 - mean_absolute_error: 0.3606 - val_loss: 0.4860 - val_mean_absolute_error: 0.4669
Epoch 56/150
1/1 [==============================] - 0s 238ms/step - loss: 0.3606 - mean_absolute_error: 0.3777 - val_loss: 0.4653 - val_mean_absolute_error: 0.4683
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 256ms/step - loss: 0.3145 - mean_absolute_error: 0.3624 - val_loss: 0.4820 - val_mean_absolute_error: 0.4938
Epoch 102/150
1/1 [==============================] - 0s 253ms/step - loss: 0.3888 - mean_absolute_error: 0.3905 - val_loss: 0.6130 - val_mean_absolute_error: 0.5454
Epoch 103/150
1/1 [==============================] - 0s 250ms/step - loss: 0.2513 - mean_absolute_error: 0.3346 - val_loss: 0.6733 - val_mean_absolute_error: 0.5823
Epoch 104/150
1/1 [==============================] - 0s 220ms/step - loss: 0.2558 - mean_absolute_error: 0.3217 - val_loss: 0.5940 - val_mean_absolute_error: 0.5356
Epoch 105/150
1/1 [==============================] - 0s 234ms/step - loss: 0.2511 - mean_absolute_error: 0.3350 - val_loss: 0.4900 - val_mean_absolute_error: 0.4820
Epoch 106/150
1/1 [==============================] - 0s 241ms/step - loss: 0.2505 - mean_absolute_error: 0.3261 - val_loss: 0.6827 - val_mean_absolute_error: 0.5844
Epoch 107/150
1/1 [=====

1/1 [==============================] - 0s 143ms/step - loss: 0.6513 - mean_absolute_error: 0.5603
val performance = [0.6512600183486938, 0.5603277683258057]
test performance =  [0.46756961941719055, 0.43298718333244324]


**CNN model**

In [251]:
CONV_WIDTH = 7
CONV_LAYER_NO = 1
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -(CONV_WIDTH*CONV_LAYER_NO-CONV_LAYER_NO+1):, :]),
] + [
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)) for i in range(CONV_LAYER_NO)
] + [
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [252]:
history = compile_and_fit(multi_conv_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

Epoch 1/150
1/1 [==============================] - 0s 470ms/step - loss: 1.0050 - mean_absolute_error: 0.7914 - val_loss: 0.6874 - val_mean_absolute_error: 0.6102
Epoch 2/150
1/1 [==============================] - 0s 233ms/step - loss: 0.5963 - mean_absolute_error: 0.6344 - val_loss: 0.7866 - val_mean_absolute_error: 0.6464
Epoch 3/150
1/1 [==============================] - 0s 224ms/step - loss: 0.5530 - mean_absolute_error: 0.6037 - val_loss: 0.8586 - val_mean_absolute_error: 0.6705
Epoch 4/150
1/1 [==============================] - 0s 238ms/step - loss: 0.4524 - mean_absolute_error: 0.5313 - val_loss: 0.9557 - val_mean_absolute_error: 0.6794
Epoch 5/150
1/1 [==============================] - 0s 217ms/step - loss: 0.5059 - mean_absolute_error: 0.5033 - val_loss: 0.8652 - val_mean_absolute_error: 0.6664
Epoch 6/150
1/1 [==============================] - 0s 232ms/step - loss: 0.6835 - mean_absolute_error: 0.4833 - val_loss: 0.9671 - val_mean_absolute_error: 0.7168
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 233ms/step - loss: 0.2727 - mean_absolute_error: 0.3715 - val_loss: 0.5443 - val_mean_absolute_error: 0.5502
Epoch 52/150
1/1 [==============================] - 0s 237ms/step - loss: 0.2557 - mean_absolute_error: 0.3692 - val_loss: 0.4732 - val_mean_absolute_error: 0.4823
Epoch 53/150
1/1 [==============================] - 0s 227ms/step - loss: 0.2596 - mean_absolute_error: 0.3457 - val_loss: 0.4532 - val_mean_absolute_error: 0.4771
Epoch 54/150
1/1 [==============================] - 0s 244ms/step - loss: 0.2931 - mean_absolute_error: 0.3664 - val_loss: 0.5545 - val_mean_absolute_error: 0.5318
Epoch 55/150
1/1 [==============================] - 0s 229ms/step - loss: 0.4835 - mean_absolute_error: 0.4146 - val_loss: 0.7480 - val_mean_absolute_error: 0.6094
Epoch 56/150
1/1 [==============================] - 0s 239ms/step - loss: 0.5110 - mean_absolute_error: 0.4211 - val_loss: 0.6086 - val_mean_absolute_error: 0.5403
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 231ms/step - loss: 0.2860 - mean_absolute_error: 0.3448 - val_loss: 0.6047 - val_mean_absolute_error: 0.5503
Epoch 102/150
1/1 [==============================] - 0s 248ms/step - loss: 0.2598 - mean_absolute_error: 0.3416 - val_loss: 0.6674 - val_mean_absolute_error: 0.6012
Epoch 103/150
1/1 [==============================] - 0s 245ms/step - loss: 0.2754 - mean_absolute_error: 0.3296 - val_loss: 0.7924 - val_mean_absolute_error: 0.6508
Epoch 104/150
1/1 [==============================] - 0s 208ms/step - loss: 0.2482 - mean_absolute_error: 0.3247 - val_loss: 0.5554 - val_mean_absolute_error: 0.5424
Epoch 105/150
1/1 [==============================] - 0s 255ms/step - loss: 0.2802 - mean_absolute_error: 0.3406 - val_loss: 0.6086 - val_mean_absolute_error: 0.5600
Epoch 106/150
1/1 [==============================] - 0s 224ms/step - loss: 0.2607 - mean_absolute_error: 0.3284 - val_loss: 0.5136 - val_mean_absolute_error: 0.5107
Epoch 107/150
1/1 [=====

1/1 [==============================] - 0s 156ms/step - loss: 0.6336 - mean_absolute_error: 0.5727
val performance = [0.6336462497711182, 0.5727380514144897]
test performance =  [0.341795414686203, 0.380831778049469]


In [253]:
multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

1/1 [==============================] - 0s 150ms/step - loss: 0.7126 - mean_absolute_error: 0.5858
val performance = [0.7125796675682068, 0.5857947468757629]
test performance =  [0.46217647194862366, 0.41654306650161743]


**performance**

In [254]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [268]:
multi_window.compa(elman_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

column : 클로로필-a
NSE
0.5572028945791789
PBIAS
5.092466724025235


In [265]:
multi_window.compa(gru_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

column : 클로로필-a
NSE
0.607064267805028
PBIAS
4.376361308081233


In [273]:
multi_window.compa(multi_lstm_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

column : 클로로필-a
NSE
0.5285753135500753
PBIAS
6.557161290132607


In [297]:
multi_window.compa(multi_conv_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

column : 클로로필-a
NSE
0.548359255697247
PBIAS
5.712080688899332


In [259]:
multi_window.plot24(gru_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …